In [85]:
import time,os,glob
import TAD_SIM as ts
import numpy as np
import cPickle as pickle
reload(ts)


<module 'TAD_SIM' from 'TAD_SIM.py'>

In [120]:
files_ = glob.glob('save_folder'+os.sep+'cell_id_*__chr_cutoff_200.0__miss_rate_0.0325__num_TADs_100__num_chr_23__num_hom_2__num_label_*.pkl')
map_ = [int(fl.split('num_label_')[-1].split('__')[0].split('.pkl')[0]) for fl in files_]
file_sets = ts.partition_map(files_,map_)
num_labeled = np.unique(map_)
perc_errs_tot_first=[]
perc_errs_tot_sep=[]
for files_ in file_sets:
    perc_errs_first,perc_errs_sep = [],[]
    for fl in files_:
        #print fl
        dic = pickle.load(open(fl,'r'))
        perc_err_sep = float(dic['incorr_sep'])/(dic['incorr_sep']+dic['corr_sep'])*100
        perc_err_first = float(dic['interp_first'])/(dic['interp_first']+dic['incorr_first'])*100
        perc_errs_sep.append(perc_err_sep)
        perc_errs_first.append(perc_err_first)
    perc_errs_tot_first.append(perc_errs_first)
    perc_errs_tot_sep.append(perc_errs_sep)
perc_errs_tot_sep200=perc_errs_tot_sep
num_labeled200 = num_labeled

In [121]:
files_ = glob.glob('save_folder'+os.sep+'cell_id_*__chr_cutoff_100.0__miss_rate_0.0325__num_TADs_100__num_chr_23__num_hom_2__num_label_*.pkl')
map_ = [int(fl.split('num_label_')[-1].split('__')[0].split('.pkl')[0]) for fl in files_]
file_sets = ts.partition_map(files_,map_)
num_labeled = np.unique(map_)
perc_errs_tot_first=[]
perc_errs_tot_sep=[]
for files_ in file_sets:
    perc_errs_first,perc_errs_sep = [],[]
    for fl in files_:
        #print fl
        dic = pickle.load(open(fl,'r'))
        perc_err_sep = float(dic['incorr_sep'])/(dic['incorr_sep']+dic['corr_sep'])*100
        perc_err_first = float(dic['interp_first'])/(dic['interp_first']+dic['incorr_first'])*100
        perc_errs_sep.append(perc_err_sep)
        perc_errs_first.append(perc_err_first)
    perc_errs_tot_first.append(perc_errs_first)
    perc_errs_tot_sep.append(perc_errs_sep)
perc_errs_tot_sep100=perc_errs_tot_sep
num_labeled100 = num_labeled

In [122]:
import matplotlib.pyplot as plt
#plt.errorbar(num_labeled,map(np.mean,perc_errs_tot_first),yerr=map(np.std,perc_errs_tot_first))
plt.errorbar(num_labeled100,map(np.mean,perc_errs_tot_sep100),yerr=map(np.std,perc_errs_tot_sep100),fmt='o-')
plt.errorbar(num_labeled200,map(np.mean,perc_errs_tot_sep200),yerr=map(np.std,perc_errs_tot_sep200),fmt='o-')
plt.xlim([11,23])
plt.show()

In [95]:
files_ = glob.glob('save_folder'+os.sep+'cell_id_*__chr_cutoff_100.0__miss_rate_0.0325__num_TADs_100__num_chr_23__num_hom_2__num_label_18.pkl')
for fl in files_:
    os.remove(fl)

In [523]:
import time
import TAD_SIM as ts
import numpy as np
reload(ts)

<module 'TAD_SIM' from 'TAD_SIM.py'>

In [1039]:
#simulate a cell
start = time.time()
chrters=ts.chromosomes(nuc_dia=10000,pixel_sz=100,nchr=46,plt_val=False,return_im=False,cutoff=500.)
print "Simulated Territories. Time(s): "+str(time.time()-start)


Simulated Territories. Time(s): 34.6710000038


In [1040]:
start = time.time()
cell=[]
for chrter in chrters:
    tads_=ts.TAD_generator(chrter,noTADs=50,udist=-0.44276236166846844,sigmadist=0.57416477624326434,nuc_dia=10000,pix_sz=100)
    cell.append(tads_)
cell=np.array(cell)
print "Simulated cell. Time(s): "+str(time.time()-start)

Simulated cell. Time(s): 4.18700003624


In [1041]:
reload(ts)
fig,ax=None,None
cols = ['r','g','b','y','m','c','w','k']
for ipts,pts in enumerate(cell[:5]):
    x,y,z = np.array(pts,dtype=float).T
    fig,ax=ts.plot3D(x,y,z,fig,ax,c=cols[ipts%len(cols)])
plt.axis('equal')
plt.show()

In [1042]:
reload(ts)
hybes=ts.code_encoder_rep(nchr=23,ntads=50,nlabel_=16,no_hom=2)
hybes_points,tot_ground_truth = ts.simulated_imdata(hybes,cell,err_rate=0.032504222398951552,sigma=50.)

No. of tads:
[100]
No. of chrms labeled/hybe:
[13 14 15 16 17 18 19] 15.9722222222 1.02702675263
No. of collisions (pairs of TADs appearing in the same hybe in repeat):70


In [1043]:
reload(ts)
goods,bads,chromosome_ids_all = ts.decoder_rep(hybes_points,hybes,tot_ground_truth,
                                               n_chr=23,cutoff_sameTAD=200.,num_TADs=50,frac_TADs=1.8)
print bads/float(goods+bads)*100
for iter_ in range(10):
    goods,bads,chromosome_ids_all = ts.refine_decoder_rep(hybes_points,hybes,chromosome_ids_all,tot_ground_truth,
                       n_chr=23,noTads=50,fr_nn=1.8,cutoff_sameTAD=200.)
    print bads/float(goods+bads)*100

21.4637421665
5.3715308863
5.21486123545
5.34914950761


KeyboardInterrupt: 

In [1051]:
tot_ground_truth_ = map(lambda x: np.array(x,dtype=int)%23, tot_ground_truth)

pts_all = ts.partition_map(ts.flatten(hybes_points),ts.flatten(tot_ground_truth_))
counter=0
for pred,truth,pts in zip(chromosome_ids_all,tot_ground_truth_,hybes_points):
    truth_,pred_=np.array(truth),np.array(pred)
    dif = pred_==truth_
    if np.prod(dif)==0:
        mis_pairs = zip(pred_[dif==False],np.array(truth_)[dif==False])
        for id_mis in range(len(mis_pairs)):
            mis_pair = mis_pairs[id_mis]
            chr_pred = mis_pair[0]
            chr_truth = mis_pair[1]
            pt_mis = pts[dif==False][id_mis]
            plt.close('all')
            counter+=1
            fig,ax=None,None
            chr_truth_,chr_pred_ = chr_truth,chr_pred
            x,y,z = np.array(pts_all[chr_truth_],dtype=float).T
            fig,ax=ts.plot3D(x,y,z,fig,ax,c='g')
            chr_truth_+=23
            x,y,z = np.array(pts_all[chr_pred_],dtype=float).T
            fig,ax=ts.plot3D(x,y,z,fig,ax,c='r')
            chr_pred_+=23
            x,y,z = pt_mis
            fig,ax=ts.plot3D(x,y,z,fig,ax,c='b',s=200)
            plt.axis('equal')
            plt.show()
            if counter>20:
                assert 0==1

AssertionError: 

In [1064]:
code.shape

(3003L, 15L)

In [1075]:
code
from scipy.spatial.distance import pdist
pdist(code[:3],'hamming')*15


array([ 2.,  2.,  2.])

In [1118]:
np.sum(pdist(code[np.random.choice(range(len(code)),23,replace=False)],'hamming'))

114.26666666666667

In [1115]:
ind_eq_sp = np.array(np.round(np.linspace(0,len(code)-1,23)),dtype=int)
np.sum(pdist(code[ind_eq_sp],'hamming'))

108.93333333333332

In [1126]:
np.sum(code[np.random.choice(range(len(code)),23,replace=False)],axis=0)

array([15, 15, 18, 15, 17, 16, 11, 17, 13, 14, 16, 14, 17, 16, 16])

In [1149]:
code_ = code[np.random.randint(len(code))]
dists = np.sum(code!=code_,axis=1)


array([ 4,  4,  6, ...,  8,  8, 10])

In [1241]:
def hamming_distance(codei,codej):
    return np.sum(codei!=codej)
def simulate_codes(all_codes,dst_min=6):
    codes=[]
    keep_codes = np.array(all_codes)
    while len(keep_codes)>0:
        #print len(codes)
        ich = np.random.randint(0,len(keep_codes))
        codes.append(keep_codes[ich])
        keep_codes_=[]
        for code_i in keep_codes:
            Good=True
            for code_j in codes:
                if hamming_distance(code_i,code_j)<dst_min:
                    Good=False
                    break
            if Good:
                keep_codes_.append(code_i)
        keep_codes=keep_codes_
    return np.array(codes)

In [1224]:
sim = simulate_codes(code,dst_min=6)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


KeyboardInterrupt: 

In [1243]:
lens = np.array(map(len,sims))
print np.max(lens)
max_sims =[sim for sim in sims if len(sim)==np.max(lens)]
stds = [np.std(np.sum(sim,axis=0)) for sim in max_sims]
sim = max_sims[np.argmin(stds)]

46


In [1248]:
np.sum(sim,axis=0)

array([29, 28, 29, 29, 28, 29, 29, 29, 30, 28, 29, 28, 29, 27, 30, 29])

In [1258]:
sim.shape

(46L, 16L)

In [1259]:
sim_rep = np.concatenate([sim[:23].T,sim[23:46].T])

In [1298]:
plt.hist(pdist(sim_,'hamming')*16)
plt.show()

In [1323]:
plt.hist(np.sum(sim_rep,axis=1))
plt.show()

In [ ]:
#############Simulation

In [1396]:
import itertools
combs = list(itertools.combinations(range(16),10))
code = ts.combs_to_code(combs)

In [ ]:
sims = [] 
for sim_n in range(100):
    print sim_n
    sims.append(simulate_codes(code))

In [1415]:
sims_=np.array(sims_)
min_ham_dists = np.array([np.min(pdist(sim_,'hamming')) for sim_ in sims_])
print np.max(min_ham_dists)*sim_.shape[-1]
sim_maxs = sims_[min_ham_dists==np.max(min_ham_dists)]
std_min = np.argmin([np.std(np.sum(sim_,axis=0)) for sim_ in sim_maxs])
sim_ = sim_maxs[std_min]
print np.sum(sim_,axis=0)
sim1,sim2=split_codes(sim_,50000)
print np.sum(sim1,axis=0)
print np.sum(sim2,axis=0)

6.0
[28 28 28 29 29 29 29 30 29 28 28 29 28 29 29 30]
[14 14 14 14 14 15 15 16 14 14 14 14 14 14 15 15]
[14 14 14 15 15 14 14 14 15 14 14 15 14 15 14 15]


In [2]:
import TAD_SIM_codes as tsc
sims = tsc.multi_codes(nhybes=12,ntads=5,num_codes=46,n_sim=1000,verbose=True)

Computing combinations...
Computing maximum spearation simulation...
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
2

In [3]:
sim1,sim2 = tsc.pick_best_sim_and_splitin2(sims,n_split=100000)

4.0
[19 20 19 19 19 19 19 18 19 20 20 19]
[10 10 10 10 10  9  9  9  9 10 10  9]
[ 9 10  9  9  9 10 10  9 10 10 10 10]


In [5]:
import numpy as np
sim1 = np.concatenate([sim1,sim1],axis=-1)
sim2 = np.concatenate([sim2,sim2],axis=-1)

In [16]:
reload(tsc)
sim2_tads_,col_ns_ = tsc.multi_min_collisions(sim1,sim2,niter=10,n_iter=1000,nTADperChr=2)

No. col. start:5246088
No. col. end:86
No. col. start:6303879
No. col. end:86
No. col. start:11539565
No. col. end:87
No. col. start:7361648
No. col. end:87
No. col. start:5778046
No. col. end:86
No. col. start:7354478
No. col. end:87
No. col. start:5773945
No. col. end:86
No. col. start:6824554
No. col. end:86
No. col. start:6825582
No. col. end:86
No. col. start:6294648
No. col. end:90


In [17]:
sim1_tad = np.cumsum(sim1,axis=-1)*sim1
print 'Min. col.:'+str(np.min(col_ns_))
sim2_tad=sim2_tads_[np.argmin(col_ns_)]


Min. col.:86


In [22]:
sim_tad1 = np.dstack([sim1_tad[:,:sim1_tad.shape[-1]/2],sim1_tad[:,sim1_tad.shape[-1]/2:]])
sim_tad2 = np.dstack([sim2_tad[:,:sim2_tad.shape[-1]/2],sim2_tad[:,sim2_tad.shape[-1]/2:]])

In [1802]:
sim1_tad.T[0],sim1_tad.T[8],sim2_tad.T[0],sim2_tad.T[8]

(array([1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1]),
 array([6, 0, 6, 6, 6, 6, 0, 0, 6, 0, 0, 6, 6, 0, 6, 6, 6, 0, 0, 6, 6, 0, 6]),
 array([1, 6, 3, 2, 0, 0, 5, 0, 0, 0, 7, 5, 7, 0, 6, 4, 3, 0, 1, 9, 8, 9, 0]),
 array([ 4, 10,  4,  8,  0,  0,  9,  0,  0,  0,  3,  8,  2,  0,  5,  2,  6,
         0,  6,  5,  1,  8,  0]))

In [18]:
col_mat =tsc.collisions(sim1_tad.T,sim2_tad.T,nTADperChr=2)
col_mat_sum = np.sum(col_mat,axis=-1)
col_mat_sum

array([[2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2, 2],
       [2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2],
       [1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2],
       [1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1],
       [1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2],
       [1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2],
       [1, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1],
       [2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2],
       [2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1],
       [2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 1, 2],
       [1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1],
       [2, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2]])

In [19]:
np.where(col_mat>1)

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [20]:
np.sum(col_mat_sum[col_mat_sum>1])

172

In [21]:
np.sum(col_mat_sum)

230

In [1792]:
reload(tsc)
tsc.multi_min_collisions(sim1,sim2,niter=10,n_iter=1000,nTADperChr=2);

No. col. start:370
No. col. end:306
No. col. start:376
No. col. end:306
No. col. start:399
No. col. end:306
No. col. start:360
No. col. end:306
No. col. start:392
No. col. end:306
No. col. start:386
No. col. end:306
No. col. start:440
No. col. end:306
No. col. start:410
No. col. end:306
No. col. start:377
No. col. end:306
No. col. start:415
No. col. end:306


In [1784]:
def collisions(sim1,sim2,nTADperChr=1):
    if nTADperChr==1:
        col_mat = np.zeros([len(sim1),len(sim2),len(sim1[0])],dtype=int)
        for i1,sim1_ in enumerate(sim1):
            for i2,sim2_ in enumerate(sim2):
                col_mat[i1,i2] = (sim1_==sim2_)*(sim1_>0)*(sim2_>0)
        return col_mat
    if nTADperChr==2:
        col_mat = np.zeros([len(sim1)/2,len(sim2)/2,len(sim1[0])],dtype=int)
        h1,h2,n_chr = col_mat.shape
        for i1 in range(h1):
            for i2 in range(h2):
                vals = np.zeros(n_chr)
                for sim1_ in [sim1[i1],sim1[i1+h1]]:
                    for sim2_ in [sim2[i2],sim2[i2+h2]]:
                        vals = vals+(sim1_==sim2_)*(sim1_>0)*(sim2_>0)
                col_mat[i1,i2] = vals
        return col_mat

In [ ]:
def collisions(sim1,sim2):
    col_mat = np.zeros([len(sim1),len(sim2),len(sim1[0])],dtype=int)
    for i1,sim1_ in enumerate(sim1):
        for i2,sim2_ in enumerate(sim2):
            col_mat[i1,i2] = (sim1_==sim2_)*(sim1_>0)*(sim2_>0)
    return col_mat

In [1777]:
sim1_tad.T.shape

(16L, 23L)

In [1780]:
def collisions2(sim1,sim2):
    col_mat = np.zeros([len(sim1)/2,len(sim2)/2,len(sim1[0])],dtype=int)
    h1,h2,n_chr = col_mat.shape
    for i1 in range(h1):
        for i2 in range(h2):
            vals = np.zeros(n_chr)
            for sim1_ in [sim1[i1],sim1[i1+h1]]:
                for sim2_ in [sim2[i2],sim2[i2+h2]]:
                    vals = vals+(sim1_==sim2_)*(sim1_>0)*(sim2_>0)
            col_mat[i1,i2] = vals
    return col_mat

In [1783]:
col_mat = collisions2(sim1_tad.T,sim2_tad.T)
np.sum(col_mat,axis=-1)



array([[4, 4, 4, 3, 3, 3, 4, 3],
       [4, 2, 4, 3, 4, 3, 4, 4],
       [3, 4, 3, 2, 4, 4, 4, 4],
       [4, 4, 4, 4, 4, 4, 3, 3],
       [4, 4, 4, 4, 3, 2, 3, 4],
       [4, 4, 3, 4, 4, 4, 4, 3],
       [4, 4, 4, 4, 4, 4, 3, 3],
       [3, 4, 2, 4, 4, 4, 3, 4]])

In [24]:
import cPickle as pickle
pickle.dump([sim_tad1,sim_tad2],open('tad_mats_2xdublets_10.pkl','wb'))

In [1639]:
reload(tsc)
col_mat = tsc.collisions(sim1_tad.T,sim2_tad.T)
col_mat_sum = np.sum(col_mat,axis=-1)
pairs_colisions = col_mat_sum*(col_mat_sum-1)/2
prev_col = np.sum(pairs_colisions)
prev_col

1

In [1773]:
fl = r'C:\Users\Bogdan\Desktop\Crulic-The Path to Beyond.srt'
def rev_time(str_time,sec=31):
    sec = np.sum(np.array(str_time.split(':'),dtype=int)*[60*60,60,1])-sec
    return ':'.join(["{0:2d}".format(i).replace(' ','0') for i in [sec/3600,(sec%3600)/60,((sec%3600)%60)]])

fid =open(fl.replace('.srt','2.srt'),'w')
for ln in open(fl,'r'):
    if ln.count('-->'):
        ln_new = rev_time(ln[0:8])+ln[8:17]+rev_time(ln[17:17+8])+ln[17+8:]
        fid.write(ln_new)
        print ln_new
    else:
        fid.write(ln)
    

00:01:27,013 --> 00:01:30,867

00:01:54,361 --> 00:01:57,921

00:01:58,416 --> 00:02:03,285

00:02:03,923 --> 00:02:05,284

00:02:06,179 --> 00:02:10,194

00:02:12,348 --> 00:02:15,800

00:02:16,315 --> 00:02:19,430

00:02:20,152 --> 00:02:24,893

00:02:40,089 --> 00:02:44,437

00:02:45,469 --> 00:02:48,514

00:02:49,102 --> 00:02:52,019

00:02:52,677 --> 00:02:56,516

00:03:04,285 --> 00:03:06,578

00:03:07,280 --> 00:03:10,302

00:03:10,848 --> 00:03:16,544

00:03:18,867 --> 00:03:21,066

00:03:21,555 --> 00:03:24,877

00:03:25,465 --> 00:03:29,370

00:03:30,588 --> 00:03:31,878

00:03:33,468 --> 00:03:35,293

00:03:36,884 --> 00:03:38,493

00:03:39,011 --> 00:03:40,396

00:03:40,722 --> 00:03:43,830

00:03:44,643 --> 00:03:46,873

00:03:47,320 --> 00:03:49,287

00:03:49,879 --> 00:03:53,717

00:03:54,315 --> 00:03:55,999

00:03:56,587 --> 00:03:59,341

00:03:59,832 --> 00:04:02,109

00:04:02,571 --> 00:04:05,359

00:04:06,558 --> 00:04:08,588

00:04:09,131 --> 00:04:11,875

00:04:50

In [1768]:
rev_time(ln[0:8])

ValueError: invalid literal for long() with base 10: '\n'

'00,01,28,013 --> 00,01,31,867\n'

In [1769]:
'00:01:58'
str_time = '00:01:58'


'02'

In [1453]:
pairs_colisions.shape
weights = np.cumsum(weights)
        weights = weights/float(np.max(weights))
        index_pj = np.sum(np.random.rand()-weights>0)

(16L, 16L)

In [1436]:
sim1_tad,sim2_tad

(array([[ 0,  0,  0,  1,  2,  0,  3,  4,  0,  5,  6,  7,  8,  9,  0, 10],
        [ 1,  2,  3,  4,  0,  5,  6,  7,  8,  0,  0,  0,  9,  0, 10,  0],
        [ 1,  2,  3,  0,  4,  5,  0,  0,  6,  7,  0,  8,  0,  9,  0, 10],
        [ 1,  2,  3,  4,  5,  0,  0,  6,  0,  0,  7,  0,  0,  8,  9, 10],
        [ 1,  0,  0,  2,  3,  4,  5,  0,  6,  7,  8,  9,  0,  0, 10,  0],
        [ 0,  1,  2,  0,  3,  4,  5,  0,  0,  0,  6,  7,  8,  0,  9, 10],
        [ 1,  0,  2,  0,  0,  0,  3,  0,  4,  5,  6,  0,  7,  8,  9, 10],
        [ 0,  1,  0,  0,  2,  3,  0,  4,  5,  6,  7,  0,  8,  9, 10,  0],
        [ 0,  0,  1,  2,  0,  3,  4,  5,  0,  6,  7,  0,  8,  9, 10,  0],
        [ 1,  0,  2,  0,  0,  3,  4,  5,  6,  7,  0,  8,  9,  0,  0, 10],
        [ 1,  2,  0,  0,  3,  4,  5,  6,  7,  0,  0,  8,  0,  9, 10,  0],
        [ 0,  0,  1,  0,  2,  0,  3,  4,  5,  6,  0,  7,  8,  9, 10,  0],
        [ 0,  1,  0,  2,  3,  4,  5,  6,  0,  7,  8,  0,  0,  0,  9, 10],
        [ 1,  2,  3,  4,  0,  0,  5,  

In [1327]:
sim_=sims_[np.argmax([np.min(pdist(sim_,'hamming'))for sim_ in sims_])]

In [1381]:
#def split_codes(code)
sim_.shape


In [1388]:
np.sum(sim1,axis=0)

array([15, 14, 14, 14, 14, 14, 15, 14, 15, 14, 15, 14, 15, 13, 15, 15])

In [1390]:
sim1,sim2

(array([[1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1],
        [1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0],
        [1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1],
        [0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1],
        [1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1],
        [0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1],
        [1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1],
        [1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0],
        [1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1],
        [0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1],
        [1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
        [0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1],
        [0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0],
        [0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1],
        [1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0],
        [1, 0,

In [1370]:
np.min(pdist(sim2,'hamming')*16)

6.0

In [1351]:
sim.shape

(46L, 16L)

In [1328]:
plt.hist(pdist(sim_,'hamming')*16)
plt.show()

In [1319]:
sim_=sims_[np.argmin([np.std(np.sum(sim_,axis=0)) for sim_ in sims_])]

In [1330]:
np.sum(sim_[:],axis=0)

array([30, 29, 28, 28, 28, 29, 31, 29, 27, 28, 30, 29, 29, 27, 28, 30])

In [1287]:
plt.imshow(simulate_codes_max(code),interpolation='nearest')
plt.show()

In [1275]:
from scipy.spatial.distance import cdist,pdist,squareform
#plt.imshow(squareform(pdist(sim,'hamming'))*16,interpolation='nearest')
#plt.show()

In [1278]:
cdist([[1],[2]],[[3],[4],[5]]).shape

(2L, 3L)

In [ ]:
pdist(,'hamming')

In [1239]:
plt.imshow(sim,interpolation='nearest')
plt.show()

In [1238]:
np.sum(sim,axis=0)

array([29, 28, 28, 28, 28, 29, 28, 28, 27, 29, 28, 29, 26, 29, 27, 29])

In [958]:
mis_pairs
mis_pair = mis_pairs[0]

pred==mis_pair[0],truth



(array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False,  True], dtype=bool),
 [0, 2, 4, 5, 6, 8, 10, 11, 12, 14, 15, 16, 18, 20, 21])

In [949]:
truth

[0, 2, 4, 5, 6, 8, 10, 11, 12, 14, 15, 16, 18, 20, 21]

In [938]:
pts_all = ts.partition_map(ts.flatten(hybes_points),ts.flatten(chromosome_ids_all))
reload(ts)
fig,ax=None,None
cols = ['r','g','b','y','m','c','w','k']
for ipts,pts in enumerate(pts_all[:5]):
    x,y,z = np.array(pts,dtype=float).T
    fig,ax=ts.plot3D(x,y,z,fig,ax,c=cols[ipts%len(cols)])
plt.axis('equal')
plt.show()

In [772]:
chromosome_ids_all[10]

array([ 0,  2,  4,  5,  6,  8, 10, 11, 12, 14, 15, 16, 21, 20,  3], dtype=int64)

In [724]:
plt.imshow(stitches_point,interpolation='nearest')
plt.show()

In [725]:
plt.imshow(projections_point,interpolation='nearest')
plt.show()

In [735]:
float(bads)/(goods+bads)*100

3.117290872392913

In [723]:
goods,bads

(14, 0)

In [703]:
chromosome_ids_all

[array([ 0, 11,  2,  3,  4,  5,  6,  7, 10,  9, 11, 12, 13, 14,  3,  0,  1,
        13,  4,  5,  6,  7,  8,  9,  8, 12, 14, 15], dtype=int64)]

In [671]:
chrs_all = ts.partition_map(ts.flatten(hybes_points),ts.flatten(chromosome_ids_all))

In [675]:
for chr_pts in chrs_all[:6]:
    x,y,z=np.array(chr_pts).T
    plt.plot(x,y,'o')
plt.show()
plt.axis('equal')

(-0.059999999999999998,
 0.059999999999999998,
 -0.059999999999999998,
 0.059999999999999998)

In [679]:
float(bads)/(goods+bads)*100

12.286612758310872

In [622]:
['A']

((6L, 8L, 2L), (6L, 8L))

In [653]:
bad

1

In [659]:
from matplotlib import cm
plt.imshow(projections_point,interpolation='nearest',cmap=cm.gray)
plt.show()

False
False
False
False
False
False
False
True


In [596]:
len(hybes)

658

In [580]:
possible_chrs,hybes_points_ref

(array([ 1,  2,  3,  4,  5,  6, 15], dtype=int64),
 array([[-1839.73988336,  2327.8029613 ,  2409.9973354 ],
        [ -714.91872869,  -901.77718298,  2689.80791569],
        [ 4740.89207194,   151.57554373,  -122.35802804],
        [ 3931.55799354,   878.2233099 ,  -940.77447986],
        [  993.14600422, -3674.22943562,  1257.67668229],
        [-1514.23475881,   311.56860333,  3325.82359796],
        [-3302.06572251, -1078.94171652,  -171.51203268]]))

In [574]:
len(possible_chrs_hybes)

658

array([1, 1, 1, 1, 1, 1, 1])

In [581]:
possible_chrs,hybehybes[id_ref][]

(array([ 1,  2,  3,  4,  5,  6, 15], dtype=int64),
 array([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]))

In [566]:
plt.plot([np.median(min_L1_dists[pos_proj==1]) for pos_proj in possible_projections],'o-')
plt.show()

In [560]:
plt.plot(min_L1_dists,'o')
plt.show()

In [570]:
n_chr=23
chr_picks = ts.unique_classif(projections_point,conf=None)
points_identities,chr_identities = zip(*chr_picks)
#chr_identities goes from 0 to number of chromosomes is ref hybe in maximum confidence order
chromosome_ids0 = np.arange(len(points_identities))
chromosome_ids0[np.array(points_identities)]=np.array(chr_identities)
chromosome_ids = possible_chrs[chromosome_ids0]%n_chr

In [571]:
chromosome_ids,tot_ground_truth[0]

(array([ 1,  2,  3,  4,  5,  6, 15], dtype=int64), [1, 2, 3, 4, 5, 6, 15])

In [ ]:
#Decoder - Given hybes_points and hybes predict chr id
def decoder_rep(hybes_points,hybes,tot_ground_truth,n_chr=23):
    #What chromosomes appear in which hybe
    possible_chrs_hybes=[]
    for hybe in hybes:
        possible_chrs_hybes.append(np.where(hybe>0)[0])
    ##
    goods,bads=0,0
    chromosome_ids_all = []
    #Iterate through all the points in the hybes. The current hybe I call it ref hybe
    for id_ref in range(len(hybes_points)):
        ###Given id_ref hybe compute the projection space
        hybes_points_ref = hybes_points[id_ref]
        
        possible_chrs = possible_chrs_hybes[id_ref]#np.where(hybes[id_ref]>0)[0]
        
        #compute possible projections: possibble chromosome x numbe of hybes - binary
        possible_projections = np.zeros([len(possible_chrs),len(possible_chrs_hybes)],dtype=int)
        for i,chr_T in enumerate(possible_chrs):
            for j,possible_chrs_hybe in enumerate(possible_chrs_hybes):
                possible_projections[i,j]=chr_T in possible_chrs_hybe
        sum_proj = np.array([np.sum(possible_projections,axis=1)])
        #sum_proj[sum_proj==0]=1
        possible_projections_ = possible_projections*1./sum_proj.T #the normalized projection space
        
        ###Compute 
        projections_point = []
        for point in hybes_points_ref:
            min_L1_dists=[]#distances to nearest neighbors across hybes for point
            for hybe_point in hybes_points:
                difs = point - hybe_point
                #min_L1_dist = np.min(np.sum(np.abs(difs),axis=-1))
                min_L1_dist = np.min(np.sqrt(np.sum(difs**2,axis=-1)))
                min_L1_dists.append(min_L1_dist)
            min_L1_dists = np.array(min_L1_dists)#nearest neighbour distance across hybes for point in reference hybe

            projection = np.dot(possible_projections_,min_L1_dists)
            projections_point.append(projection)
        ##After computing a no of candidate chromosomes x no of points weight matrix projections_point
        ## Decide on best assigment.
        chr_picks = unique_classif(projections_point,conf=None)
        
        
        points_identities,chr_identities = zip(*chr_picks)
        #chr_identities goes from 0 to number of chromosomes is ref hybe in maximum confidence order
        chromosome_ids0 = np.arange(len(points_identities))
        chromosome_ids0[np.array(points_identities)]=np.array(chr_identities)
        chromosome_ids = possible_chrs[chromosome_ids0]%n_chr
        chromosome_ids_all.append(chromosome_ids)
        #chromosome_ids is chromosome prediction (0-22) in order of the points in ref hybe.
        #Compare to ground truth calculated during simulation of imaging data.
        non_deg_poss=np.array(tot_ground_truth[id_ref])%n_chr
        good = np.sum(non_deg_poss==chromosome_ids) #up to degeneracy due to homologous chromosomes
        bad = np.sum(non_deg_poss!=chromosome_ids)
        goods+=good
        bads+=bad

In [629]:
proj_vec = [0.2,10,10,10]
stich_vec = [1,0,0,0]

conf(np.dstack([proj_vec,stich_vec])[0])  
        
    

('A', 9.8000000000000007)

In [627]:
proj_vec = [0.2,10,10,10]
stich_vec = [1,0,0,0]
.shape

(4L, 2L)

In [435]:
np.where(stich_vec),no_stich

((array([0], dtype=int64),), 1)

In [188]:
num_TADs=100

hybes_rep = np.array(hybes,dtype=float)
hybes_rep[hybes_rep==0]=0
for shift in range(num_TADs):
    hybes_rep = ((hybes_rep+shift)%num_TADs+1)*(hybes_rep>0)
    hybes_rep = np.array(np.roll(hybes_rep,hybes_rep.shape[0]/2,axis=0),dtype=int)
    if np.sum((hybes_rep==hybes)*(hybes>0)):
        break
print shift
hybes_rep

6


array([[  1,   0,  79, ...,   0,   0,  82],
       [  2,   0,  80, ...,   0,   0,   0],
       [  3,   0,  81, ...,   0,  78,  83],
       ..., 
       [ 98,   0,  76, ...,   0,  77,  79],
       [ 99,   0,  77, ...,  73,   0,  80],
       [100,   0,  78, ...,   0,   0,  81]])

In [194]:
hybes_rep = np.array(hybes)
hybes_rep = hybes_rep%100
hybes_rep[hybes_rep==0]=100
hybes_rep = hybes_rep*(hybes>0)
hybes_rep

array([[  1,   1,   1, ...,   0,   0,   0],
       [  2,   2,   2, ...,   1,   1,   1],
       [  3,   3,   3, ...,   2,   2,   2],
       ..., 
       [  0,   0,   0, ...,  99,  98,  98],
       [  0,   0,   0, ...,   0,  99,  99],
       [  0,   0,   0, ..., 100, 100, 100]])

In [420]:
def code_encoder_rep(nchr=23,ntads=100,nlabel_=2,no_hom=1):
    """Master function for the encoder with repetition
    nchr is the number of *unique* - i.e. non-homologous - chromosomes
    no_hom is the number of homologous chromosomes
    nlabel is the numbr of TADs labeled in each hybe
    #Interpretation of codes: codes is number of hybe x number of chromosomes and indicates which chr is present in each hybe
    #Interpretation of hybes: hybes is number of hybe x number of chromosomes and indicates which TAD is present in each hybe
    #                         0 means chromose not appearing and if not 0 then it encodes which TAD from the chr appears
    Return hybes
    
    ###Example use:
    hybes = code_encoder_rep(nchr=23,ntads=100,nlabel_=10)
    """
    combs = list(itertools.combinations(range(nchr),nlabel_))
    nhybes = 2*int(float(nchr)*ntads/nlabel_+1)
    inds = np.array(np.round(np.linspace(0,len(combs)-1,nhybes)),dtype=int)
    combs_eq_sp = [combs[ind] for ind in inds]
    codes = ts.combs_to_code(combs_eq_sp)
    codes[:codes.shape[0]/2] = ts.patch_code(codes[:codes.shape[0]/2],target=ntads)
    codes[codes.shape[0]/2:] = ts.patch_code(codes[codes.shape[0]/2:],target=ntads)
    assert ts.test_code(codes)
    codes1 = codes[:codes.shape[0]/2]
    codes2 = codes[codes.shape[0]/2:]
    hybes1=np.cumsum(codes1,axis=0)*codes1
    hybes2=np.cumsum(codes2,axis=0)*codes2
    def shift_hybe(hybe,shift=0):
        hybes_t = np.array(hybe)
        hybes_tT = (hybes_t+shift)%ntads
        hybes_tT[hybes_tT==0]=ntads
        hybes_tT = hybes_tT*(hybes_t>0)
        return hybes_tT
    for ichr in range(hybes2.shape[-1]):
        hybes2[:,ichr]=shift_hybe(hybes2[:,ichr],shift=ichr*ntads/nchr)
    ninters = 0
    for hybe1_t in hybes1:
        for hybe2_t in hybes2:
            inters = ((hybe1_t==hybe2_t)*hybe1_t*hybe2_t)>0
            ninters_ = np.sum(inters)
            if ninters_>1:
                ninters+=ninters_*(ninters_-1)/2
    print "No. of collisions (pairs of TADs appearing in the same hybe in repeat):"+str(ninters)
    hybes = np.concatenate([hybes1,hybes2])
    hybes=np.concatenate([hybes.T]*no_hom).T
    return hybes
hybes = code_encoder_rep(nchr=23,ntads=100,nlabel_=16,no_hom=1);

No. of tads:
[200]
No. of chrms labeled/hybe:
[13 14 15 16 17 18 19] 15.9722222222 0.957024065386
No. of collisions (pairs of TADs appearing in the same hybe in repeat):75


In [422]:
hybes

array([[  1,   1,   1, ...,   0,   0,   0],
       [  0,   0,   2, ...,   1,   0,   1],
       [  2,   0,   3, ...,   2,   1,   2],
       ..., 
       [  0,   2,   0, ...,  85,  89,  93],
       [100,   3,   0, ...,   0,  90,  94],
       [  0,   4,   0, ...,  86,  91,  95]])

In [398]:
import itertools
nchr=23
ntads=100
nlabel_=16
no_hom=1
    combs = list(itertools.combinations(range(nchr),nlabel_))
    nhybes = 2*int(float(nchr)*ntads/nlabel_+1)
    inds = np.array(np.round(np.linspace(0,len(combs)-1,nhybes)),dtype=int)
    combs_eq_sp = [combs[ind] for ind in inds]
    codes = ts.combs_to_code(combs_eq_sp)
    codes[:codes.shape[0]/2] = ts.patch_code(codes[:codes.shape[0]/2],target=ntads)
    codes[codes.shape[0]/2:] = ts.patch_code(codes[codes.shape[0]/2:],target=ntads)
    assert ts.test_code(codes)
    codes1 = codes[:codes.shape[0]/2]
    codes2 = codes[codes.shape[0]/2:]
    hybes1=np.cumsum(codes1,axis=0)*codes1
    hybes2=np.cumsum(codes2,axis=0)*codes2
    def shift_hybe(hybe,shift=0):
        hybes_t = np.array(hybe)
        hybes_tT = (hybes_t+shift)%ntads
        hybes_tT[hybes_tT==0]=ntads
        hybes_tT = hybes_tT*(hybes_t>0)
        return hybes_tT
    for ichr in range(hybes2.shape[-1]):
        hybes2[:,ichr]=shift_hybe(hybes2[:,ichr],shift=ichr*ntads/nchr)
    ninters = 0
    for hybe1_t in hybes1:
        for hybe2_t in hybes2:
            inters = ((hybe1_t==hybe2_t)*hybe1_t*hybe2_t)>0
            ninters_ = np.sum(inters)
            if ninters_>1:
                ninters+=ninters_*(ninters_-1)/2
    print "No. of collisions (pairs of TADs appearing in the same hybe in repeat):"+str(ninters_)

ninters

No. of tads:
[200]
No. of chrms labeled/hybe:
[14 15 16 17 18 19] 15.9722222222 0.832870241698


49

1262

In [379]:
ninters

55

In [356]:
hybes_t

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,   0,   0,  63,
        64,   0,   0,  65,  66,   0,  67,   0,   0,   0,  68,  69,   0,
         0,   0,  70,   0,  71,  72,  73,   0,  74,   0,   0,   0,   0,
        75,   0,  76,   0,   0,   0,  77,  78,  79,  80,  81,   0,  82,
         0,   0,  83,   0,  84,   0,  85,  86,   0,  87,   0,   0,  88,
        89,   0,   0,  90,  91,   0,  92,   0,   0,  93,   0,  94,  95,
         0,  96,   0,   0,  97,   0,  98,   0,   0,   0,  99,   0, 100,   0])

In [344]:
map(lambda hb: np.unique(hb)[1:],hybes1)

[array([1]),
 array([1, 2]),
 array([1, 2, 3]),
 array([1, 2, 3, 4]),
 array([2, 3, 4, 5]),
 array([3, 4, 5, 6]),
 array([3, 4, 5, 6, 7]),
 array([3, 4, 5, 6, 7, 8]),
 array([4, 5, 6, 7, 9]),
 array([ 5,  6,  7,  8, 10]),
 array([ 5,  6,  7,  8,  9, 11]),
 array([ 6,  7,  8,  9, 10, 12]),
 array([ 7,  8,  9, 10, 13]),
 array([ 6,  7,  8,  9, 10, 11, 14]),
 array([ 7,  9, 10, 11, 12, 15]),
 array([ 6,  8,  9, 10, 11, 12, 13, 16]),
 array([ 7,  8,  9, 10, 11, 13, 17]),
 array([ 9, 10, 11, 12, 13, 14, 18]),
 array([ 8, 11, 12, 13, 14, 15, 19]),
 array([12, 13, 14, 15, 20]),
 array([ 9, 10, 12, 13, 14, 15, 16, 21]),
 array([10, 11, 13, 14, 15, 16, 17, 22]),
 array([11, 12, 13, 14, 15, 16, 17, 23]),
 array([12, 13, 14, 15, 17, 24]),
 array([14, 15, 16, 18, 25]),
 array([13, 14, 15, 16, 17, 18, 19, 26]),
 array([14, 16, 17, 18, 20, 27]),
 array([15, 17, 18, 19, 21, 28]),
 array([16, 17, 19, 20, 21, 22, 29]),
 array([17, 18, 20, 21, 22, 23, 30]),
 array([18, 19, 20, 21, 22, 23, 24, 31]),
 arr

In [341]:
class Found(Exception): pass

    
hybe1_t,hybe2_t=hybes1[6],hybes2[6]
try:
    for hybe1_t in hybes1:
        for hybe2_t in hybes2:
            inters = ((hybe1_t==hybe2_t)*hybe1_t*hybe2_t)>0
            if len(inters)>1:
                raise Found
except Found:
    print np.where(inters)

np.where(inters)[0][:1]

(array([ 0,  1,  2,  3,  4,  5,  6,  8, 10, 12, 14, 15], dtype=int64),)


array([0], dtype=int64)

In [333]:
hybe1_t*hybe2_t

array([20,  0, 12, 25, 15,  0,  0,  0,  0,  9,  0,  0])

In [307]:
hybes2

array([[1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 2, 1, 2, 2, 0, 0, 0, 0, 0],
       [0, 2, 2, 3, 0, 3, 0, 2, 0, 0, 2, 1],
       [0, 3, 3, 0, 2, 0, 3, 0, 2, 1, 3, 0],
       [2, 4, 0, 4, 3, 4, 0, 0, 3, 2, 0, 2],
       [3, 5, 0, 0, 4, 0, 4, 3, 0, 0, 0, 3],
       [4, 0, 4, 5, 5, 0, 5, 0, 0, 3, 4, 0],
       [0, 0, 5, 0, 0, 0, 0, 4, 4, 4, 0, 4],
       [5, 0, 0, 0, 0, 5, 0, 5, 5, 5, 5, 5]])

In [255]:
hybes_ = hybes2[:,0]
permut = np.random.permutation(ntads)

In [271]:
for ilast in range(hybes2.shape[0]):
    permut = np.random.permutation(ntads)
    hybes_ = hybes2[:,ilast]
    for ip,p in enumerate(permut):
        hybes_[hybes_==(ip+1)]=(p+1)*(len(permut)+1)
    hybes_/=(len(permut)+1)
map(list,hybes2)

[[4, 5, 10, 2, 7, 2, 0, 5, 0, 6, 0, 0, 3, 0, 10, 1, 0, 1, 1, 1, 1, 0, 1],
 [8, 6, 2, 7, 0, 9, 2, 6, 2, 1, 2, 10, 0, 0, 4, 0, 1, 2, 0, 2, 0, 1, 0],
 [7, 0, 3, 0, 5, 6, 10, 0, 0, 0, 5, 9, 0, 6, 5, 0, 2, 0, 2, 0, 2, 2, 0],
 [9, 10, 1, 0, 0, 0, 3, 3, 8, 0, 6, 7, 5, 0, 0, 2, 3, 3, 3, 0, 3, 0, 2],
 [10, 0, 0, 10, 6, 8, 9, 2, 0, 8, 10, 0, 0, 10, 3, 3, 4, 0, 4, 3, 4, 3, 0],
 [6, 0, 0, 0, 9, 0, 8, 1, 5, 5, 4, 8, 1, 1, 0, 4, 0, 4, 5, 0, 0, 4, 3],
 [0, 3, 8, 3, 10, 0, 1, 0, 1, 0, 0, 0, 7, 7, 0, 5, 5, 5, 6, 0, 5, 5, 4],
 [5, 4, 4, 1, 2, 0, 6, 0, 10, 4, 0, 6, 8, 5, 1, 0, 0, 6, 7, 4, 0, 0, 5],
 [1, 8, 5, 9, 0, 10, 0, 7, 7, 0, 0, 5, 4, 4, 9, 6, 6, 7, 8, 5, 0, 6, 6],
 [0, 1, 6, 6, 8, 5, 0, 4, 3, 0, 3, 0, 0, 3, 7, 7, 0, 0, 0, 6, 6, 7, 7],
 [0, 7, 0, 4, 3, 1, 5, 10, 0, 7, 9, 2, 10, 9, 8, 0, 7, 0, 0, 7, 0, 0, 0],
 [0, 2, 0, 8, 0, 3, 0, 8, 4, 9, 7, 1, 6, 0, 6, 8, 0, 8, 0, 8, 7, 8, 8],
 [0, 0, 7, 5, 4, 7, 7, 0, 0, 3, 0, 0, 9, 0, 2, 0, 8, 0, 0, 0, 8, 9, 9],
 [3, 9, 9, 0, 1, 4, 4, 9, 9, 2, 8, 3, 0, 2, 0, 9, 

[[1, 2, 6, 1, 8, 4, 0, 2, 0, 4, 0, 0, 5, 0, 5, 1, 0, 1, 1, 1, 1, 0, 1],
 [7, 7, 5, 10, 0, 1, 1, 3, 7, 10, 8, 5, 0, 0, 8, 0, 1, 2, 0, 2, 0, 1, 0],
 [4, 0, 9, 0, 7, 5, 7, 0, 0, 0, 4, 6, 0, 1, 10, 0, 2, 0, 2, 0, 2, 2, 0],
 [2, 9, 7, 0, 0, 0, 3, 4, 9, 0, 3, 7, 9, 0, 0, 2, 3, 3, 3, 0, 3, 0, 2],
 [3, 0, 0, 2, 4, 3, 2, 7, 0, 5, 9, 0, 0, 8, 7, 3, 4, 0, 4, 3, 4, 3, 0],
 [10, 0, 0, 0, 6, 0, 5, 10, 4, 6, 10, 10, 6, 2, 0, 4, 0, 4, 5, 0, 0, 4, 3],
 [0, 5, 1, 9, 1, 0, 4, 0, 6, 0, 0, 0, 1, 4, 0, 5, 5, 5, 6, 0, 5, 5, 4],
 [5, 6, 8, 8, 9, 0, 8, 0, 10, 2, 0, 9, 8, 7, 2, 0, 0, 6, 7, 4, 0, 0, 5],
 [6, 3, 10, 3, 0, 2, 0, 6, 2, 0, 0, 8, 4, 9, 9, 6, 6, 7, 8, 5, 0, 6, 6],
 [0, 8, 3, 5, 5, 10, 0, 9, 8, 0, 2, 0, 0, 6, 3, 7, 0, 0, 0, 6, 6, 7, 7],
 [0, 10, 0, 4, 2, 6, 10, 1, 0, 3, 5, 3, 10, 10, 6, 0, 7, 0, 0, 7, 0, 0, 0],
 [0, 1, 0, 6, 0, 8, 0, 5, 3, 9, 6, 1, 7, 0, 4, 8, 0, 8, 0, 8, 7, 8, 8],
 [0, 0, 2, 7, 10, 9, 9, 0, 0, 1, 0, 0, 3, 0, 1, 0, 8, 0, 0, 0, 8, 9, 9],
 [8, 4, 4, 0, 3, 7, 6, 8, 5, 8, 1, 4, 0, 3, 0, 9,

In [16]:
hybes=ts.code_encoder(nchr=23,ntads=100,nlabel_=16,no_hom=2)
id1,id2 =0,3
in1 = np.array([id1 in np.where(hybe>0)[0] for hybe in hybes])
in2 = np.array([id2 in np.where(hybe>0)[0] for hybe in hybes])
np.sum(in1*in2)

No. of tads:
[100]
No. of chrms labeled/hybe:
[14 15 16 17] 15.9722222222 0.611742098487


70

In [ ]:
import time,os,glob
import TAD_SIM as ts
import numpy as np
import cPickle as pickle
reload(ts)

dic_parms = {'num_chr':23,'num_hom':2,'num_TADs':100,'chr_cutoff':0.,'num_label':16,'miss_rate':0.0325,'cell_id':0}
#simulate a chr ter
for chr_cutoff in [0.,200.,400.,100.,300.,500.][::-1]:
    dic_parms['chr_cutoff']=chr_cutoff
    start = time.time()
    chrters=ts.chromosomes(nuc_dia=10000,pixel_sz=100,nchr=dic_parms['num_chr']*dic_parms['num_hom'],plt_val=False,
                           return_im=False,cutoff=dic_parms['chr_cutoff'])
    print "Simulated chr teritorries. Time(s): "+str(time.time()-start)

    #simulate single cells
    for cell_id in range(10):
        start = time.time()
        cell=[]
        for chrter in chrters:
            tads_=ts.TAD_generator(chrter,noTADs=dic_parms['num_TADs'],udist=-0.44276236166846844,sigmadist=0.57416477624326434,
                                   nuc_dia=10000,pix_sz=100)
            cell.append(tads_)
        cell=np.array(cell)
        dic_parms['cell_id'] = cell_id
        print "Simulated cell. Time(s): "+str(time.time()-start)

        for num_label in [12,14,16,20,22]:
            #decide on encoder
            start = time.time()
            dic_parms['num_label'] = num_label
            hybes=ts.code_encoder(nchr=dic_parms['num_chr'],ntads=dic_parms['num_TADs'],nlabel_=dic_parms['num_label'],
                                  no_hom=dic_parms['num_hom'])
            print "Created encoder. Time(s): "+str(time.time()-start)

            #simulate data from a cell
            start = time.time()
            im_data,truth=ts.simulated_imdata(hybes, cell, err_rate=dic_parms['miss_rate'])
            print "Simulated data. Time(s): "+str(time.time()-start)

            #decode up to homology
            start = time.time()
            corr,incorr,interp=ts.decoder(im_data,hybes,truth,n_chr=dic_parms['num_chr'])
            corr_first,incorr_first,interp_first = map(np.array,interp),corr,incorr

            print "First decoding. %Incorrect: "+str(incorr/float(corr+incorr)*100)
            for iter_ in range(3):
                corr,incorr,interp=ts.refine_decoder(im_data,hybes,interp,truth,n_chr=dic_parms['num_chr'],
                                                     noTads=dic_parms['num_TADs'],fr_nn=0.8)
                print "Refining decoding - step "+str(iter_)+". %Incorrect: "+str(incorr/float(corr+incorr)*100)
            corr_last,incorr_last,interp_last = map(np.array,interp),corr,incorr
            print "Decoded up to homology. Time(s): "+str(time.time()-start)

            #decode hom chrs
            start = time.time()
            corrsep,incorrsep,interpsep = ts.separator(im_data,interp,truth,num_chr=dic_parms['num_chr'],num_hom=dic_parms['num_hom'])

            print "Separated homologs. Time(s): "+str(time.time()-start)
            dic_keep={}
            dic_keep['cell']=cell
            dic_keep['encoder']=hybes
            dic_keep['sim_data']=im_data
            dic_keep['truth']=truth

            dic_keep['interp_first']=interp_first
            dic_keep['corr_first']=corr_first
            dic_keep['incorr_first']=incorr_first

            dic_keep['interp_last']=interp_last
            dic_keep['corr_last']=corr_last
            dic_keep['incorr_last']=incorr_last

            dic_keep['interp_sep']=interpsep
            dic_keep['corr_sep']=corrsep
            dic_keep['incorr_sep']=incorrsep
            dic_keep['dic_parms']=dic_parms

            filename = '__'.join([key+'_'+str(dic_parms[key]) for key in np.sort(dic_parms.keys())])+'.pkl'
            save_folder = 'save_folder'
            if not os.path.exists(save_folder):
                os.mkdir(save_folder)
            pickle.dump(dic_keep,open(save_folder+os.sep+filename,'wb'))

Simulated chr teritorries. Time(s): 31.7419998646
Simulated cell. Time(s): 7.85100007057
No. of tads:
[100]
No. of chrms labeled/hybe:
[ 9 10 11 12 13 14] 11.9791666667 0.735685149292
Created encoder. Time(s): 0.344000101089
Simulated data. Time(s): 0.0149998664856
First decoding. %Incorrect: 18.6681715576
Refining decoding - step 0. %Incorrect: 16.2528216704


In [5]:
dic_parms = {'num_chr':23,'num_hom':2,'num_TADs':100,'chr_cutoff':0.,'num_label':16,'miss_rate':0.0325}

In [16]:
6*10*6

360

In [17]:
import os

In [3]:
import time
import TAD_SIM as ts
import numpy as np
reload(ts)


#simulate a cell
start = time.time()
chrters=ts.chromosomes(nuc_dia=10000,pixel_sz=100,nchr=46,plt_val=False,return_im=False,cutoff=0.)


cell=[]
for chrter in chrters:
    tads_=ts.TAD_generator(chrter,noTADs=100,udist=-0.44276236166846844,sigmadist=0.57416477624326434,nuc_dia=10000,pix_sz=100)
    cell.append(tads_)
cell=np.array(cell)
print "Simulated cell. Time(s): "+str(time.time()-start)

#decide on encoder
start = time.time()
hybes=ts.code_encoder(nchr=23,ntads=100,nlabel_=16,no_hom=2)
print "Created encoder. Time(s): "+str(time.time()-start)

#simulate data from a cell
start = time.time()
im_data,truth=ts.simulated_imdata(hybes, cell, err_rate=0.0325)
dic_keep['cell']=cell
dic_keep['encoder']=hybes
dic_keep['sim_data']=im_data
dic_keep['truth']=truth
print "Simulated data. Time(s): "+str(time.time()-start)

#decode up to homology
start = time.time()
corr,incorr,interp=ts.decoder(im_data,hybes,truth,n_chr=23)
dic_keep['interp_first']=interp
dic_keep['corr_first']=corr
dic_keep['incorr_first']=incorr

print "First decoding. %Incorrect: "+str(incorr/float(corr+incorr)*100)
for iter_ in range(3):
    corr,incorr,interp=ts.refine_decoder(im_data,hybes,interp,truth,n_chr=23,noTads=100,fr_nn=0.8)
    print "Refining decoding - step "+str(iter_)+". %Incorrect: "+str(incorr/float(corr+incorr)*100)
dic_keep['interp_last']=interp
dic_keep['corr_last']=corr
dic_keep['incorr_last']=incorr
print "Decoded up to homology. Time(s): "+str(time.time()-start)

#decode hom chrs
start = time.time()
corr,incorr,interpsep = ts.separator(im_data,interp,truth,num_chr=23,num_hom=2)
dic_keep['interpsep']=interpsep
dic_keep['corrsep']=corr
dic_keep['incorrsep']=incorr
print "Separated homologs. Time(s): "+str(time.time()-start)


Simulated chr teritorries. Time(s): 24.736000061
Simulated cell. Time(s): 4.37999987602
No. of tads:
[100]
No. of chrms labeled/hybe:
[15 16 17 18] 15.9722222222 0.600282855138
Created encoder. Time(s): 0.0780000686646
Simulated data. Time(s): 0.0
First decoding. %Incorrect: 3.96128741841
Refining decoding - step 0. %Incorrect: 1.57551204141
Refining decoding - step 1. %Incorrect: 1.53049741166
Refining decoding - step 2. %Incorrect: 1.53049741166
Decoded up to homology. Time(s): 85.0750000477


NameError: global name 'num_chr' is not defined

In [4]:
reload(ts)
start = time.time()
corr,incorr,interpsep = ts.separator(im_data,interp,truth,num_chr=23,num_hom=2)
dic_keep['interpsep']=interpsep
dic_keep['corrsep']=corr
dic_keep['incorrsep']=incorr
print "Separated homologs. Time(s): "+str(time.time()-start)

Separated homologs. Time(s): 18.3229999542


In [125]:
reload(ts)
interpsep = ts.separator(im_data,interp,num_chr=23,num_hom=2)

In [124]:
def compare_to_truth(interpsep,truth,num_hom=2):
    pairs = []
    for hb in range(len(interpsep)):
        pairs.extend(zip(interpsep[hb],truth[hb]))
    chrs_pairs = ts.partition_map(pairs,[pair[-1]%num_chr for pair in pairs])

    import itertools
    permuts = np.array(list(itertools.permutations(range(num_hom))))
    
    total = np.sum(map(len,truth))
    correct = 0
    interpsep_ = map(np.array,interpsep)
    for chr_pairs in chrs_pairs:
        chr_pairs = np.array(chr_pairs,dtype=int)

        id_chrs_truth = np.unique(chr_pairs[:,1])

        perm_values = []
        for permut in permuts:
            id_chrs_data = id_chrs_truth[permut]
            perm_values.append(np.sum([np.sum((chr_pairs[:,0]==idd)*(chr_pairs[:,1]==idt)) for idd,idt in zip(id_chrs_data,id_chrs_truth)]))
        best_permut = permuts[np.argmax(perm_values)]
        id_chrs_data = id_chrs_truth[permut]
        for idd,idt in zip(id_chrs_data,id_chrs_truth):
            interpsep_ = [interpsep__==3for interpsep__ in interpsep_]
            
        correct+=np.max(perm_values)
    return correct,total-correct
compare_to_truth(interpsep,truth,num_hom=2)

(4402, 44)

In [130]:
arr=np.array([1,2,3,4,4,4,2,2,2,3])
arr[arr==2]=-1
arr

array([ 1, -1,  3,  4,  4,  4, -1, -1, -1,  3])

In [120]:
id_chrs_truth

array([ 0, 23])

In [103]:
num_chr=23
#num_hom=2


In [45]:
point_col = ts.flatten(im_data)
chr_col = ts.flatten(interp)
point_part = ts.partition_map(point_col,chr_col)

chr0pts = np.array(point_part[0])
import matplotlib.pyplot as plt
plt.plot(chr0pts[:,0],chr0pts[:,1],'o')


point_col = ts.flatten(im_data)
chr_col = ts.flatten(interp2)
point_part = ts.partition_map(point_col,chr_col)

chr0pts = np.array(point_part[0])
import matplotlib.pyplot as plt
plt.plot(chr0pts[:,0],chr0pts[:,1],'o')
plt.show()

In [52]:
corr,corr2,corr3,corr4

(4259, 4388, 4389, 4392)

In [50]:
corr+incorr,corr2+incorr2

(4439, 4439)

In [36]:
possible_chrs_hybes=[]
for hybe in hybes:
    possible_chrs_hybes.append(np.where(hybe>0)[0])

In [83]:
truth,interp_hom

([[0,
   1,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36],
  [0,
   1,
   2,
   3,
   5,
   6,
   7,
   8,
   9,
   16,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   28,
   29,
   30,
   31,
   32,
   39,
   41,
   42,
   43,
   44,
   45],
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   15,
   16,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   38,
   39,
   41,
   42,
   43,
   44,
   45],
  [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   9,
   15,
   16,
   17,
   18,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   32,
   38,
   39,
   41,
   43,
   44,
   45],
  [0,
   1,
   3,
   4,
   5,
   6,
   8,
   9,
   11,
   15,
   16,
   17,
   19,
   20,
   22,
   23,
   24,
   26,
   27,
   28,
   29,
   31,
   32,
   33,
   34,
   38,
   3

In [77]:
reload(ts)
def separator(im_data,interp,num_chr=23,num_hom=2):
    im_data_enhanced = map(list,im_data)
    for ihybe in range(len(im_data_enhanced)):
        im_data_enhanced_hybe = im_data_enhanced[ihybe]
        for ipoint in range(len(im_data_enhanced_hybe)):
            point = im_data_enhanced_hybe[ipoint]
            point = list(point)+[ihybe,ipoint]
            im_data_enhanced_hybe[ipoint] = point
        im_data_enhanced[ihybe] = im_data_enhanced_hybe
    #iterate through chromosomes to patition the data
    pts_chrs,chr_hybes = [],[]
    for pts_hybe,chr_hybe in zip(im_data_enhanced,interp):
        pts_chr_ = ts.partition_map(pts_hybe,chr_hybe)
        chr_hybe_ = np.unique(chr_hybe)
        pts_chrs.extend(pts_chr_)
        chr_hybes.extend(chr_hybe_)
    pts_partitioned = ts.partition_map(pts_chrs,chr_hybes)
    chrs_ids = np.unique(chr_hybes)

    interp_hom = map(np.array,interp)
    #iterate through chromosome ids
    for chrs_id,pts_partitioned_ in zip(chrs_ids,pts_partitioned):
        #chrs_id = chrs_ids[1]
        #pts_partitioned_ = pts_partitioned[1]

        id_hybe_start = np.argsort(map(len,pts_partitioned_))[-1]
        pts_start = pts_partitioned_[id_hybe_start]
        assert len(pts_start)==num_hom
        #split_chr = split_dic[chrs_id]
        #for ipt,pt in enumerate(pts_start):
        #    split_chr[ipt].append(pt)

        pts_tobeasigned = list(pts_partitioned_)
        pts_tobeasigned.pop(id_hybe_start)

        chr_estim = [[list(val)+[ival]for ival,val in enumerate(pts_partitioned_[id_hybe_start])]]

        for pts_hybe in pts_tobeasigned:
            mean_dists_hybe = []
            chr_estim_flat = np.array(ts.flatten(chr_estim))
            split_chr = ts.partition_map(chr_estim_flat,chr_estim_flat[:,-1])
            for pt in pts_hybe:
                mean_dists = [np.median([np.sqrt(np.sum((pt[:3]-pt_t[:3])**2)) for pt_t in split_]) for split_ in split_chr]
                #mean_dists has num of homolog dists
                mean_dists_hybe.append(mean_dists)
            picks = ts.unique_classif(mean_dists_hybe)
            chr_estim.append([list(pts_hybe[pick[0]][:])+[pick[1]] for pick in picks])

        chr_estim_new = chr_estim
        while True:
            no_flips,chr_estim_new = refine_separator(chr_estim_new)
            if no_flips==0:
                break
        #Return the sames as interp(chromosme_id_all) but dealt with degeneracy
        for pt in ts.flatten(chr_estim_new):
            interp_hom[pt[3]][pt[4]]+=pt[-1]*num_chr
    return interp_hom
def refine_separator(chr_estim):
    chr_estim_flat = np.array(ts.flatten(chr_estim))
    split_chr = ts.partition_map(chr_estim_flat,chr_estim_flat[:,-1])
    chr_estim_new=[]
    for pts_hybe in chr_estim:
        mean_dists_hybe = []
        for pt in pts_hybe:
            mean_dists = [np.median([np.sqrt(np.sum((pt[:3]-pt_t[:3])**2)) for pt_t in split_]) for split_ in split_chr]
            #mean_dists has num of homolog dists
            mean_dists_hybe.append(mean_dists)
        picks = ts.unique_classif(mean_dists_hybe)
        chr_estim_new.append([list(pts_hybe[pick[0]][:])+[pick[1]] for pick in picks])

    chr_estim_flat = np.array(ts.flatten(chr_estim))
    chr_estim_flat_new = np.array(ts.flatten(chr_estim_new))  
    reind_old = np.argsort(chr_estim_flat[:,0])
    reind_new = np.argsort(chr_estim_flat_new[:,0])
    pos_dif = np.where(chr_estim_flat[reind_old,-1]!=chr_estim_flat_new[reind_new,-1])[0]
    no_flips = len(pos_dif)
    return no_flips,chr_estim_new

In [80]:
np.unique(ts.flatten(interp_hom))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45], dtype=int64)

In [75]:
interp_hom = map(np.array,interp)
for pt in ts.flatten(chr_estim_new):
    interp_hom[pt[3]][pt[4]]+=pt[-1]*num_chr

In [76]:
interp_hom

[array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,  0, 24,
         2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13], dtype=int64),
 array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 16, 18, 19, 20, 21, 22,  0, 24,
         2,  3,  5,  6,  7,  8,  9, 16, 18, 19, 20, 21, 22], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 15, 16, 18, 19, 20, 21, 22,  0,
        24,  2,  3,  4,  5,  6,  7,  8, 15, 16, 18, 19, 20, 21, 22], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 15, 16, 17, 18, 20, 21, 22,  0,
        24,  2,  3,  4,  5,  6,  7,  9, 15, 16, 18, 20, 21, 22], dtype=int64),
 array([ 0,  1,  3,  4,  5,  6,  8,  9, 11, 15, 16, 17, 19, 20, 22,  0, 24,
         3,  4,  5,  6,  8,  9, 10, 11, 15, 16, 17, 19, 20, 22], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  8, 10, 11, 14, 16, 17, 19, 21, 22,  0,
        24,  2,  3,  4,  5,  6,  8, 10, 14, 16, 17, 19, 21, 22], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  9, 11, 12, 13, 14, 17, 18, 19, 21,  0,
  

In [35]:
ts.unique_classif([[1.4075008068002779,0.25378325414531328]])

[(0, 1)]

In [53]:
chr_estim_flat = np.array(ts.flatten(chr_estim))
chr_estim_flat_new = np.array(ts.flatten(chr_estim_new))
np.sum(chr_estim_flat[:,-1]!=chr_estim_flat_new[:,-1])
pos_dif = np.where(chr_estim_flat[:,-1]!=chr_estim_flat_new[:,-1])[0]

In [59]:
reind_old = np.argsort(chr_estim_flat[:,0])
reind_new = np.argsort(chr_estim_flat_new[:,0])
pos_dif = np.where(chr_estim_flat[reind_old,-1]!=chr_estim_flat_new[reind_new,-1])[0]

In [60]:
pos_dif

array([], dtype=int64)

In [54]:
import matplotlib.pyplot as plt
flat_ = chr_estim_flat
for i in [0,1]:
    x,y,z = flat_[flat_[:,-1]==i,:3].T
    plt.plot(x,y,'o')
for pos in pos_dif:
    x,y,z = flat_[pos,:3].T
    plt.plot([x],[y],'ro')
plt.show()

In [56]:
chr_estim_flat_new[pos],chr_estim_flat[pos]

(array([  0.09105405,   0.75620262,  -0.18331434,  88.        ,
         18.        ,   1.        ,   1.        ]),
 array([  0.53733261,  -0.31869798,   0.588431  ,  88.        ,
          0.        ,   0.        ]))

In [51]:
import matplotlib.pyplot as plt
flat_ = chr_estim_flat_new
for i in [0,1]:
    x,y,z = flat_[flat_[:,-1]==i,:3].T
    plt.plot(x,y,'o')
plt.show()

In [8]:
split_chr

[[array([0.3496437678138829, -0.23852404398710134, 0.74997597970551455, 99, 0], dtype=object)],
 [array([-0.22296103835982559, 0.45835915908589808, -0.2090516553835064, 99,
         14], dtype=object)]]

In [12]:
pts_partitioned_[2]

[array([0.18553405305226198, -0.52716214232740188, 0.5549414906245359, 2, 0], dtype=object),
 array([-0.088050912958783131, 0.74077052807902488, -0.25216482427925263, 2,
        16], dtype=object)]